<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png"/>

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   






# Un breve tour por BERT

BERT, modelo auto-encoding, es probablemente uno de los transformers más conocidos. 

En su pre-entrenamiento se utilizaron dos estrategias: predicción de tokens enmascarados y predecir si dos oraciones son consecutivas. Es un modelo bidireccional. BERT fue pre-entrenado con una gran colección de textos (BookCorpus y Wikipedia). 

BERT puede ser facilmente ajustado para cualquier tarea de NLP, añadiendo simplemente un capa de salida adecuada para la tarea. En su momento, BERT fue capaz de superar los resultados de los sistemas desarrollados hasta el momento en muchas tareas de NLP como la clasificación de textos o el reconocimiento de entidades. Sin embargo, aunque BERT fue entrenado utilizando una tarea de modelado de lenguaje al predecir tokens enmascarados, BERT no es un modelo apropiado para tareas de generación de texto. 

El siguiente [paper](#https://aclanthology.org/N19-1423/) describe en detalle la arquitectura de BERT.


En los siguientes notebooks, aprenderemos a ajustar BERT para distintas tareas de NLP. HuggingFace proporciona algunas clases pre-definidas que pueden er utilizados para tareas específicas. 

El primer paso será instalar la librería transformers de HuggingFace.

In [24]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Reviewing BERT parameters

We can see some properties:
- *max_position_embeddings*:512, tamaño de la secuencia más larga que BERT puede procesar. 
- *hidden_size*: 768. Dimensión de los vectores. 
- *vocab_size*: el vocabulario de BERT tiene un total de 30,522 tokens únicos.
- *num_attention_heads*: 12, es el número de cabezas de atención en cada capa de atención.
- *num_hidden_layers*: 12, número de capas ocultas.
- *pad_token_id*: 0, es el identificador para el token de padding. 
- *position_embedding_type*: en BERT, se utiliza una posición absoluta para representar los tokens. .

Puedes encontrar más información en el siguiente [link](#https://huggingface.co/docs/transformers/model_doc/bert).

In [25]:
from transformers import BertModel, BertConfig

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model from the bert-base-uncased style configuration
model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config

print(configuration)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



## BertForNextSentencePrediction
Como se ha dicho antes, predecir si dos oraciones eran consecutivas fue una de las estrategias utilizadas para pre-entrenar BERT. 
La librería transformers nos proporciona una clase que nos permite utilizar  esta estrategia sobre pares de oraciones. La clase recibe como entrada dos oraciones, y devuelve la probabilidad de que esas dos oraciones puedan ocurrir juntas. 
Antes de utilizar la clase, debemos cargar el tokenizador asociado con el modelo BERT. Esta tokenizador nos permite representar los textos en el formato adecuado que necesita BERT (tokens_ids, token_type_ids, attention_mask).


In [26]:
from transformers import BertTokenizer, BertForNextSentencePrediction

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForNextSentencePrediction.from_pretrained("bert-base-uncased")

first_sentence = "Leonardo da Vinci was an Italian man who lived in the time of the Renaissance."
second_sentence = "He is famous for his paintings."

# the tokenizer transform the two sentences to the input format that BERT needs. 
encoding = tokenizer(first_sentence, second_sentence, return_tensors="pt")

print(encoding)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': tensor([[  101, 14720,  4830, 23765,  2001,  2019,  3059,  2158,  2040,  2973,
          1999,  1996,  2051,  1997,  1996,  8028,  1012,   102,  2002,  2003,
          3297,  2005,  2010,  5265,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
         1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}


Como ya explicamos en notebooks anteriores, el tokenizador devuelve un diccionario con tres campos: input_ids, token_type_ids, attention_mask. 

In [27]:
print(encoding.input_ids)       #ids of the tokens
print(encoding.token_type_ids) #0's for tokens of the first sentence, 1's for tokens of the second one
print(encoding.attention_mask)  #1's for tokens, 0's for padding. 


tensor([[  101, 14720,  4830, 23765,  2001,  2019,  3059,  2158,  2040,  2973,
          1999,  1996,  2051,  1997,  1996,  8028,  1012,   102,  2002,  2003,
          3297,  2005,  2010,  5265,  1012,   102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
         1, 1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])


Ahora podemos utilizar la clase **BertForNextSentencePrediction** para calcular la probabilidad.
Aplicamos el modelo sobre el encoding de las dos oraciones. 

El modelo devuelve un objeto vector (Tensor), **logists**, cuyo primer elemento es la probabilidad de que las dos oraciones pueden ocurrir juntas. Podemos ver que obtenemos un valor alto alto que indica que las dos oraciones pueden ocurrir juntas.


In [28]:
outputs = model(**encoding) 
logits = outputs.logits
print(type(logits))
print(logits)

# assert logits[0, 0] < logits[0, 1]  # next sentence was random

<class 'torch.Tensor'>
tensor([[ 5.9158, -5.4032]], grad_fn=<AddmmBackward0>)


Sin embargo, en el siguiente ejemplo, la probabilidad es negativa, lo que indica que ambas oraciones no es probable que aparezcan juntas. 

In [29]:
first_sentence = "Leonardo da Vinci was an Italian man who lived in the time of the Renaissance."
second_sentence = "Pizza is an Italian food that was created in Italy."

# the tokenizer transform the two sentences to the input format that BERT needs. 
encoding = tokenizer(first_sentence, second_sentence, return_tensors="pt")

outputs = model(**encoding)
logits = outputs.logits
print(logits)
# assert logits[0, 0] < logits[0, 1]  # next sentence was random

tensor([[-0.1121,  2.5781]], grad_fn=<AddmmBackward0>)


## TFBertForMaskedLM

Vamos a probar también la otra estrategia utilizada para pre-entrenar BERT: predecir un token enmascarado. 
La librería **transformers** ya incluye una clase, TFBertForMaskedLM, que implementa la tarea. En este caso, la entrada será una oración donde un token ha sido reemplazado por una máscara [𝑀𝐴𝑆𝐾]. 

In [30]:
from transformers import TFBertForMaskedLM
import tensorflow as tf


model = TFBertForMaskedLM.from_pretrained("bert-base-uncased")
encoding = tokenizer("This [MASK] is beautiful.", return_tensors="tf")

print(encoding)

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


{'input_ids': <tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[ 101, 2023,  103, 2003, 3376, 1012,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


In [36]:
output = model(**encoding)
logits = output.logits
print(logits)


tf.Tensor(
[[[ -6.45293    -6.4097757  -6.42824   ...  -5.812307   -5.594598
    -3.9613857]
  [-12.233643  -12.233464  -12.092814  ... -11.535319  -10.2975025
    -5.544696 ]
  [ -7.1337223  -7.2510386  -7.133424  ...  -6.402859   -6.3338857
    -3.5274947]
  ...
  [-10.7301    -11.032369  -11.200561  ... -10.39187    -9.92152
    -5.0652137]
  [-12.400526  -12.120652  -12.676919  ...  -9.836598  -10.383312
    -7.837446 ]
  [-11.660168  -11.908185  -11.60688   ... -10.483731   -8.913956
    -7.989448 ]]], shape=(1, 7, 30522), dtype=float32)


Después de aplicar el modelo sobre la entrada codificada, este devuelve un tensor  con dimensión (1,7,30522).

- donde 7 es el número de tokens en la oración de entrada: "$[CLS]$ This $[MASK]$ is beautiful. $[SEP]$". Tiene 7 tokens al incluir los tokens especiales $[CLS]$ y $[SEP]$.
- 30.522 es el tamaño de vocabulario de BERT.




La salida está en **logits[0]**, es un tensor (vector) de forma 7 x 30522, cada fila representa un token y cada columna representa una de las 30.522 tokens del vocabulario de BERT. BERT ha calculado la probabilidad del token para cada uno de los tokens del vocabulario, así en la fila i y columna j, el tensor contiene la probabilidad de que el token representado en la fila i sea el token j del vocabulario. 
Por ejemplo, $logits[0][2]$ va a contener el vector de probabilidades que el modelo ha calculado para el token $[MASK]$ en la oración "$[CLS]$ This $[MASK]$ is beautiful.$[SEP]$". La probabilidad más alta se corresponderá con el token predecido para $[MASK]$. 

En primer lugar, vamos a calcular de forma automática el índice de $[MASK]$ en el vocabulario y en la oración de entrada. 


In [32]:
print(tokenizer.mask_token_id)
# check that the decoding of this id is MASK
print(tokenizer.decode(tokenizer.mask_token_id))

103
[ M A S K ]


Por tanto, $[MASK]$ tiene el índice 103 en el vocabulario de BERT. 
Ahora vamos a obtener el índice de $[MASK]$ en la oración de entrada: 

In [37]:
print(encoding.input_ids)
print(type(encoding.input_ids))
print(type(encoding.input_ids.numpy()))

# convert from Tensor to list: tensor -> numpy -> list
input_sentence_list = (encoding.input_ids.numpy().tolist())
print(input_sentence_list)
input_sentence_list = input_sentence_list[0]
print(input_sentence_list)


tf.Tensor([[ 101 2023  103 2003 3376 1012  102]], shape=(1, 7), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'numpy.ndarray'>
[[101, 2023, 103, 2003, 3376, 1012, 102]]
[101, 2023, 103, 2003, 3376, 1012, 102]


¿Cuál es la posición del token mask_token_id en la oración? 

In [38]:
# find the index 
mask_token_index = input_sentence_list.index(tokenizer.mask_token_id)
mask_token_index
print("posición de [MASK] en la oración:", mask_token_index)


posición de [MASK] en la oración: 2


Ahora vamos a obtener el tensor que ha calculado BERT para ese token:

In [40]:
output = logits[0]
print(print(type(output)))
# from Tensor to numpy to list
output_list = output.numpy().tolist()
predicted_list_mask = output_list[mask_token_index]
print("predictions for mask:", output_list[mask_token_index])



<class 'tensorflow.python.framework.ops.EagerTensor'>
None
predictions for mask: [-7.133722305297852, -7.251038551330566, -7.133423805236816, -7.145671844482422, -7.254054069519043, -7.453934192657471, -7.352295875549316, -7.362424373626709, -7.202183246612549, -7.319637775421143, -7.083044052124023, -7.312304973602295, -7.35496187210083, -7.220702648162842, -7.471529006958008, -7.275533199310303, -7.3792195320129395, -7.34043025970459, -7.360307693481445, -7.133652210235596, -7.373200416564941, -7.376169681549072, -7.195675849914551, -7.2177653312683105, -7.180471897125244, -7.381508827209473, -7.1689605712890625, -7.197506904602051, -7.261927604675293, -7.319146156311035, -7.140125751495361, -7.474775314331055, -7.178462505340576, -7.270917892456055, -7.294820308685303, -7.299783706665039, -7.247615814208984, -7.470158100128174, -7.21692419052124, -7.332930564880371, -7.298222541809082, -7.195003509521484, -7.172225475311279, -7.160667419433594, -7.190794467926025, -7.145098686218262

Calculamos el índice cuyo valore es el mayor del tesnro. Para eso usamos la función **argmax**:

In [42]:
predicted_id_token_mask = tf.math.argmax(predicted_list_mask, axis=-1)
predicted_id_token_mask

<tf.Tensor: shape=(), dtype=int64, numpy=2173>

Para conocer el token que ha inferido el modelo, necesitamos utilizar la función **decode**: 


In [43]:
tokenizer.decode(predicted_id_token_mask)

'p l a c e'

Hemos recuperado el token, transformando el tensor a una lista y operando con sus funciones. Sin embargo, en lugar de hacer esto, es recomendable utilizar la operaciones para trabajar con Tensor (un tensor es una matriz multidimensional). 
Ver: https://www.tensorflow.org/guide/tensor

Por ejemplo, for obtener el índice de $[MASK]$ en la oración, debemos hacer lo siguiente: 

In [44]:
mask_token_index = tf.where((encoding.input_ids == tokenizer.mask_token_id)[0])
print(mask_token_index)

tf.Tensor([[2]], shape=(1, 1), dtype=int64)


In [45]:
selected_logits = tf.gather_nd(logits[0], indices=mask_token_index)

# we have to take the maximum value
predicted_id_token_mask = tf.math.argmax(predicted_list_mask, axis=-1)
#finally we decode to obtain the word
tokenizer.decode(predicted_id_token_mask)

'p l a c e'

### Ejercicio: 
Prueba tu mismo/a con otras oraciones: "Please, read this $[MASK]$."

In [47]:
inputs = tokenizer("Please, read this [MASK].", return_tensors="tf")
logits = model(**inputs).logits

mask_token_index = tf.where((inputs.input_ids == tokenizer.mask_token_id)[0])
print(mask_token_index)
selected_logits = tf.gather_nd(logits[0], indices=mask_token_index)

# we have to take the maximum value
predicted_id_token_mask = tf.math.argmax(selected_logits, axis=-1)
#finally we decode to obtain the word
tokenizer.decode(predicted_id_token_mask)

tf.Tensor([[5]], shape=(1, 1), dtype=int64)


'letter'

## BertForTokenClassification

La librería **transformes** proporciona también una clase que permite utilizar el modelo BERT para la tarea de reconocimiento de entidades (NER). NER se puede ver como una tarea donde el objetivo es clasificar cada token en una oración, y las clases siguen el estándar IOB (O no entidad, I- token interno de entidad). 

Vamos a cargar un modelo de BERT que ha sido ajustaedo a la tarea de NER. 

In [52]:
from transformers import BertTokenizer, BertForTokenClassification
import torch

tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = BertForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

inputs = tokenizer(
    "HuggingFace is a company based in Paris and New York", add_special_tokens=False, return_tensors="pt"
)

# we use the model to infer, so we need to disable gradient calculation
with torch.no_grad():
    logits = model(**inputs).logits
# logits

In [53]:
predicted_token_class_ids = logits.argmax(-1)
print(predicted_token_class_ids)
# Note that tokens are classified rather then input words which means that
# there might be more predicted token classes than words.
# Multiple token classes might account for the same word
predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]
predicted_tokens_classes
# ['O', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'I-LOC', 'I-LOC'] 

tensor([[0, 6, 6, 6, 0, 0, 0, 0, 0, 8, 0, 8, 8]])


['O',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'I-LOC',
 'O',
 'I-LOC',
 'I-LOC']


BERT necesita codificar la posición de cada token. Esta posición es necesaria para representar cómo un token en un posición afecta a otro token en una posición diferente. **BERT utiliza la posición absoluta de cada token** (desde 1 hasta la longitud máxima de la secuencia). Por este motivo, **el padding en BERT debe aplicarse a la derecha**.

En los siguientes notebooks, aprenderemos cómo ajustar BERT para la tarea de clasificación de textos. También estudiaremos otros modelos basados en BERT:  Distilbert, RoBERTa, etc. 

